In [1]:
from datasets import load_dataset
import dotenv
import os
import requests

dotenv.load_dotenv(override=True)


DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY")

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

deepseek_endpoint = "https://api.deepseek.com/chat/completions"
openai_endpoint = "https://api.openai.com/v1/chat/completions"

api_url = deepseek_endpoint
header = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
}
def get_cot(question: str):
    for i in range(3):
        try:
            payload = {
                "model": "deepseek-reasoner",
                "messages": [
                    {"role": "user", "content": "Hello!"},
                ],
            }

            resp = requests.post(api_url, headers=header, json=payload)
            return resp.json()
        except Exception as e:
            print("error, retrying...")

c:\Users\pbcon\miniconda3\envs\distill\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
resp = get_cot("hello")
print(resp)
print(resp["choices"][0]["message"]["content"])

In [ ]:
import datasets
upload = "pbcong/gsm8k_cot"
ans_list = []


def make_cot_dataset():
    data = load_dataset("pbcong/gsm8k_step_by_step", split = "train")
    data = data.shuffle()[:3]
    data = datasets.Dataset.from_dict(data)
    # print(data)
    for item in data:
        question = item["question"]
        print(question)
        cot = get_cot(question=question)
        ans = f"<think>\n\n{cot["choices"][0]["message"]["reasoning_content"]}</think>\n\n{cot["choices"][0]["message"]["content"]}"
        ans_list.append(ans)

    data.add_column("cot_ans", ans_list)

    if upload:
        data.push_to_hub(upload)

In [ ]:
make_cot_dataset()

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

data = load_dataset("pbcong/gsm8k_step_by_step")
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
print("loading model")
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", device_map="cuda"
)

loading model


c:\Users\pbcon\miniconda3\envs\distill\Lib\site-packages\huggingface_hub\file_download.py:651: UserWarning: Not enough free disk space to download the file. The expected file size is: 3554.21 MB. The target location C:\Users\pbcon\.cache\huggingface\hub\models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B\blobs only has 2803.35 MB free disk space.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
from transformers import Seq2SeqTrainer, TrainingArguments, DataCollatorForSeq2Seq
import evaluate
import numpy as np
import torch

def prepare_dataset(data):
    def tokenize_function(examples):
        return tokenizer(
            examples['question'] + ' ' + examples['answer'],
            truncation=True,
            max_length=512,
            padding='max_length'
        )
    
    # Tokenize the dataset
    tokenized_dataset = data.map(
        tokenize_function,
        batched=True,
        remove_columns=data['train'].column_names
    )
    
    return tokenized_dataset
tokenized_dataset = prepare_dataset(data["train"])
metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


train_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=1000,
    eval_steps=1000,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    save_total_limit=3,
)
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer
)
trainer = Seq2SeqTrainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")